In [42]:
from keras.models import load_model
import sys
import numpy as np
import glob
import os

if ('../../PythonClient/' not in sys.path):
    sys.path.insert(0, '../../PythonClient/')
from AirSimClient import *

# If None, then the model with the lowest validation loss from training will be used

#MODEL_PATH = 'model/models/model_model.01-0.0227733.h5'
MODEL_PATH = 'model/models/model_model.05-0.0032585.h5'

if (MODEL_PATH == None):
    models = glob.glob('model/models/*.h5') 
    best_model = max(models, key=os.path.getctime)
    MODEL_PATH = best_model
    
print('Using model {0} for testing.'.format(MODEL_PATH))

Using model model/models/model_model.05-0.0032585.h5 for testing.


In [43]:
model = load_model(MODEL_PATH)

client = CarClient()
client.confirmConnection()
client.enableApiControl(True)
car_controls = CarControls()
print('Connection established!')

Waiting for connection: 
Connection established!


In [44]:
car_controls.steering = 0
car_controls.throttle = 0
car_controls.brake = 0

image_buf = np.zeros((1, 59, 255, 3))
state_buf = np.zeros((1,4))

In [ ]:
def get_image():
    image_response = client.simGetImages([ImageRequest(0, AirSimImageType.Scene, False, False)])[0]
    image1d = np.fromstring(image_response.image_data_uint8, dtype=np.uint8)
    image_rgba = image1d.reshape(image_response.height, image_response.width, 4)
    
    return image_rgba[76:135,0:255,0:3].astype(float)

In [ ]:

while (True):
    car_state = client.getCarState()
    
    if (car_state.speed < 5):
        car_controls.throttle = 1.0
    else:
        car_controls.throttle = 0.0
    
    image_buf[0] = get_image()
    state_buf[0] = np.array([car_controls.steering, car_controls.throttle, car_controls.brake, car_state.speed])
    model_output = model.predict([image_buf, state_buf])
    car_controls.steering = round(0.5 * float(model_output[0][0]), 2)
    
    print('Sending steering = {0}, throttle = {1}'.format(car_controls.steering, car_controls.throttle))
    
    client.setCarControls(car_controls)

C:\Users\johnema\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  app.launch_new_instance()


Sending steering = 1.3, throttle = 1.0
Sending steering = 1.72, throttle = 1.0
Sending steering = 1.55, throttle = 1.0
Sending steering = 1.58, throttle = 1.0
Sending steering = 1.33, throttle = 1.0
Sending steering = 1.57, throttle = 1.0
Sending steering = -0.73, throttle = 1.0
Sending steering = -1.03, throttle = 1.0
Sending steering = -2.19, throttle = 1.0
Sending steering = -2.66, throttle = 1.0
Sending steering = -1.39, throttle = 1.0
Sending steering = -1.0, throttle = 1.0
Sending steering = -1.94, throttle = 1.0
Sending steering = -0.04, throttle = 1.0
Sending steering = 0.11, throttle = 1.0
Sending steering = -1.07, throttle = 1.0
Sending steering = -0.73, throttle = 1.0
Sending steering = -0.81, throttle = 1.0
Sending steering = -0.33, throttle = 1.0
Sending steering = -0.4, throttle = 1.0
Sending steering = 1.89, throttle = 0.0
Sending steering = 0.73, throttle = 0.0
Sending steering = 0.66, throttle = 0.0
Sending steering = 1.08, throttle = 0.0
Sending steering = 1.0, thrott